In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from keras.layers import Layer, Bidirectional, LSTM, Dense, GlobalAveragePooling1D, Dropout
from keras import Input, Model 
from keras.utils import to_categorical, plot_model

In [25]:
X_train = np.load("dataset/X_train.npy")
y_train = np.load("dataset/y_train.npy")
X_test = np.load("dataset/X_test.npy")
y_test = np.load("dataset/y_test.npy")

In [26]:
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
class DotProductAttention(Layer):
    def __init__(self):
        super(DotProductAttention, self).__init__()

    def call(self, inputs):
        query = key = value = inputs
        d_k = tf.cast(tf.shape(key)[-1], tf.float32)
        scores = tf.matmul(query, key, transpose_b=True) / tf.math.sqrt(d_k)
        weights = tf.nn.softmax(scores, axis=-1)
        return tf.matmul(weights, value)

In [28]:
input_layer = Input(shape=(X_train.shape[1], 1))  # contoh: (3000, 1)
x = Bidirectional(LSTM(64, return_sequences=True))(input_layer)
x = DotProductAttention()(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.3)(x)
x = Dense(32, activation='relu')(x)
output_layer = Dense(len(np.unique(y_train)), activation='softmax')(x)  # auto jumlah kelas

model = Model(inputs=input_layer, outputs=output_layer)

In [29]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [30]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=64,
    verbose=1
)

Epoch 1/30
1/1 [==============================] - 4s 4s/step - loss: 0.7188 - accuracy: 0.5000 - val_loss: 0.6631 - val_accuracy: 1.0000
Epoch 2/30
1/1 [==============================] - 0s 139ms/step - loss: 0.6690 - accuracy: 1.0000 - val_loss: 0.6294 - val_accuracy: 1.0000
Epoch 3/30
1/1 [==============================] - 0s 172ms/step - loss: 0.6371 - accuracy: 1.0000 - val_loss: 0.5974 - val_accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 162ms/step - loss: 0.6037 - accuracy: 1.0000 - val_loss: 0.5669 - val_accuracy: 1.0000
Epoch 5/30
1/1 [==============================] - 0s 144ms/step - loss: 0.5878 - accuracy: 1.0000 - val_loss: 0.5371 - val_accuracy: 1.0000
Epoch 6/30
1/1 [==============================] - 0s 145ms/step - loss: 0.5410 - accuracy: 1.0000 - val_loss: 0.5065 - val_accuracy: 1.0000
Epoch 7/30
1/1 [==============================] - 0s 146ms/step - loss: 0.5137 - accuracy: 1.0000 - val_loss: 0.4745 - val_accuracy: 1.0000
Epoch 8/30
1/1 [=======

In [32]:
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f"\n🎯 Test Accuracy: {acc:.4f}")


🎯 Test Accuracy: 1.0000


In [36]:
plot_model(model, to_file='model_bilstm_attention.png', show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [ ]:
model.save("bilstm_attention_ecg.h5")